##### This script takes in the V,J and CDR3 regions of TCR and creates full sequences from IMGT fasta files
- Need to fix this script - doesn't seem to be actually pulling the TCR alpha and beta sequences properly
- Also random * in there somewhere ? Need to debug this

In [ ]:
import pandas as pd
from Bio import SeqIO
import re
from pathlib import Path

TCR_dir = Path("/home/natasha/multimodal_model") / "data" / "raw" / "TCR"
raw_dir = Path("/home/natasha/multimodal_model") / "data" / "raw"

In [18]:
# 1. Load the combined IMGT AA without-gaps FASTA
in_fasta = TCR_dir / "imgt_download.fasta"

# 2. Prepare output handles
outs = {
    ("TR", "V-REGION"): open(TCR_dir / "TRV_regions.fasta","w"),
    ("TR", "J-REGION"): open(TCR_dir / "TRJ_regions.fasta","w"),
}

# 3. Iterate and filter
for rec in SeqIO.parse(in_fasta, "fasta"):
    # header is like: >accession|TRAV35*01|Homo sapiens|F|V-REGION|...
    parts = rec.description.split("|")
    if len(parts) < 6:
        continue

    gene_name = parts[1]       # e.g. "TRAV35*01" or "TRAJ40*01"
    region    = parts[4]       # e.g. "V-REGION" or "J-REGION"
    locus     = gene_name[:3]  # "TRV" or "TRA"—we’ll just check startswith "TRV" or "TRAJ"
    
    if region == "V-REGION" and gene_name.startswith("TRAV"):
        SeqIO.write(rec, outs[("TR","V-REGION")], "fasta")
    elif region == "V-REGION" and gene_name.startswith("TRBV"):
        SeqIO.write(rec, outs[("TR","V-REGION")], "fasta")
    elif region == "J-REGION" and gene_name.startswith("TRAJ"):
        SeqIO.write(rec, outs[("TR","J-REGION")], "fasta")
    elif region == "J-REGION" and gene_name.startswith("TRBJ"):
        SeqIO.write(rec, outs[("TR","J-REGION")], "fasta")

# 4. Close files
for h in outs.values():
    h.close()

print("Wrote TRV_regions.fasta and TRJ_regions.fasta")

Wrote TRV_regions.fasta and TRJ_regions.fasta


In [19]:
# import re
# import pandas as pd
# from Bio import SeqIO

# # 1) Load your Excel
# dir = Path("/home/natasha/multimodal_model") / "data" / "raw"
# df = pd.read_excel(dir / "NeoTCR data-20221220.xlsx")

# # 2) Parse the combined FASTAs into dicts
# def split_v_regions(fasta_path):
#     TRAV, TRBV = {}, {}
#     for rec in SeqIO.parse(fasta_path, "fasta"):
#         allele = rec.id.split()[0].strip()
#         if allele.startswith("TRAV"):
#             TRAV[allele] = str(rec.seq)
#         elif allele.startswith("TRBV"):
#             TRBV[allele] = str(rec.seq)
#     return TRAV, TRBV

# def split_j_regions(fasta_path):
#     TRAJ, TRBJ = {}, {}
#     for rec in SeqIO.parse(fasta_path, "fasta"):
#         allele = rec.id.split()[0].strip()
#         if allele.startswith("TRAJ"):
#             TRAJ[allele] = str(rec.seq)
#         elif allele.startswith("TRBJ"):
#             TRBJ[allele] = str(rec.seq)
#     return TRAJ, TRBJ

# TRAV, TRBV = split_v_regions(TCR_dir / "TRV_regions.fasta")
# TRAJ, TRBJ = split_j_regions(TCR_dir / "TRJ_regions.fasta")

# # 3) Helper to splice CDR3 into V and J, with logging
# def build_full_chain(v_dict, j_dict, v_allele, j_allele, cdr3):
#     v_key = str(v_allele).strip()
#     j_key = str(j_allele).strip()
    
#     V = v_dict.get(v_key)
#     J = j_dict.get(j_key)
#     if V is None:
#         print(f"⚠️ V allele not found in reference: '{v_key}'")
#     if J is None:
#         print(f"⚠️ J allele not found in reference: '{j_key}'")
#     if V is None or J is None:
#         return ""  # blank fallback

#     # cut V at last Cys
#     v_cys = V.rfind("C")
#     # find FGxG motif in J
#     m = re.search(r"[FWY][GAVLIP][G]", J)
#     j_start = m.start() if m else 0
#     if m is None:
#         print(f"⚠️ FGxG motif not found in J allele '{j_key}' – using J from pos 0")

#     return V[: v_cys+1] + cdr3 + J[j_start:]

# # 4) Apply to each row, building two new columns
# alpha_seqs = []
# beta_seqs  = []

# for _, row in df.iterrows():
#     # Strip and uppercase your inputs in case of hidden whitespace/casing
#     trav = row["TRAV"].strip().upper()
#     traj = row["TRAJ"].strip().upper()
#     trbv = row["TRBV"].strip().upper()
#     trbj = row["TRBJ"].strip().upper()
#     cdr3a = row["TRA_CDR3"].strip().upper()
#     cdr3b = row["TRB_CDR3"].strip().upper()

#     a = build_full_chain(TRAV, TRAJ, trav, traj, cdr3a)
#     b = build_full_chain(TRBV, TRBJ, trbv, trbj, cdr3b)
#     alpha_seqs.append(a)
#     beta_seqs.append(b)

# df["TCR_alpha"] = alpha_seqs
# df["TCR_beta"]  = beta_seqs

# # # 5) Save out
# # df.to_excel("NeoTCR_with_full_sequences_debug.xlsx", index=False)
# # print("Done — wrote NeoTCR_with_full_sequences_debug.xlsx")


In [23]:
import re
import pandas as pd
from pathlib import Path

# # Directories
# raw_dir = Path("/mnt/data")
# TCR_dir = Path("/mnt/data")

# Improved FASTA parser that extracts allele name from IMGT headers
def read_imgt_fasta(path, region_tag):
    """
    region_tag: "V-REGION" or "J-REGION"
    Returns dict mapping allele_name -> sequence
    """
    seqs = {}
    header = None
    seq_lines = []
    keep = False

    for line in open(path):
        line = line.rstrip()
        if line.startswith(">"):
            # save previous record
            if header and keep and seq_lines:
                seqs[allele] = "".join(seq_lines)
            # parse new header
            header = line[1:]
            parts = header.split("|")
            allele = parts[1].strip()
            region = parts[4].strip()
            keep = (region == region_tag)
            seq_lines = []
        else:
            if keep:
                seq_lines.append(line)
    # final record
    if header and keep and seq_lines:
        seqs[allele] = "".join(seq_lines)

    return seqs

# 1) Load your Excel
df = pd.read_excel(raw_dir / "NeoTCR data-20221220.xlsx")

# 2) Load and split FASTAs correctly
all_V = read_imgt_fasta(TCR_dir / "TRV_regions.fasta", "V-REGION")
all_J = read_imgt_fasta(TCR_dir / "TRJ_regions.fasta", "J-REGION")

TRAV = {a:s for a,s in all_V.items() if a.startswith("TRAV")}
TRBV = {a:s for a,s in all_V.items() if a.startswith("TRBV")}
TRAJ = {a:s for a,s in all_J.items() if a.startswith("TRAJ")}
TRBJ = {a:s for a,s in all_J.items() if a.startswith("TRBJ")}

# 3) Helper to splice CDR3 into V and J, no motif search
def build_full_chain(v_dict, j_dict, v_allele, j_allele, cdr3):
    v_key = str(v_allele).strip().upper()
    j_key = str(j_allele).strip().upper()
    
    V = v_dict.get(v_key)
    J = j_dict.get(j_key)
    if V is None:
        print(f"⚠️ V allele not found: '{v_key}'")
    if J is None:
        print(f"⚠️ J allele not found: '{j_key}'")
    if not V or not J:
        return ""  # fallback

    # cut V at last conserved Cys
    v_cys = V.rfind("C")
    if v_cys < 0:
        v_cys = len(V)  # if no Cys found, take entire V

    # splice: V up to Cys + CDR3 + full J
    return V[:v_cys+1] + cdr3.strip().upper() + J

# 4) Apply to each row
alpha_seqs = []
beta_seqs  = []

for _, row in df.iterrows():
    a = build_full_chain(TRAV, TRAJ,
                         row["TRAV"], row["TRAJ"],
                         row["TRA_CDR3"])
    b = build_full_chain(TRBV, TRBJ,
                         row["TRBV"], row["TRBJ"],
                         row["TRB_CDR3"])
    alpha_seqs.append(a)
    beta_seqs.append(b)

df["TCR_alpha"] = alpha_seqs
df["TCR_beta"]  = beta_seqs

# 5) Save out
output_path = raw_dir / "NeoTCR_with_full_sequences.xlsx"
df.to_excel(output_path, index=False)
print(f"Done — wrote {output_path}")


⚠️ V allele not found: 'TRDV3*01'
⚠️ V allele not found: 'TRAV30*01, TRAV30*02'
⚠️ V allele not found: 'TRBV6-2*01, TRBV6-3*01'
⚠️ J allele not found: 'N.A.'
⚠️ V allele not found: 'TRBV12-3*01, TRBV12-4*01'
⚠️ V allele not found: 'TRBV12-3*01, TRBV12-4*01'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found: 'N.A.'
⚠️ V allele not found:

In [ ]:
# IEDB Negatives HLA Class 1

df = pd.read_excel(raw_dir / 'IEDB' / 'IEDB_Negatives_HLA_class_I.xlsx')


# 2) Load and split FASTAs correctly
all_V = read_imgt_fasta(TCR_dir / "TRV_regions.fasta", "V-REGION")
all_J = read_imgt_fasta(TCR_dir / "TRJ_regions.fasta", "J-REGION")

TRAV = {a:s for a,s in all_V.items() if a.startswith("TRAV")}
TRBV = {a:s for a,s in all_V.items() if a.startswith("TRBV")}
TRAJ = {a:s for a,s in all_J.items() if a.startswith("TRAJ")}
TRBJ = {a:s for a,s in all_J.items() if a.startswith("TRBJ")}


alpha_seqs = []
beta_seqs  = []

for _, row in df.iterrows():
    a = build_full_chain(TRAV, TRAJ,
                         row["TRAV"], row["TRAJ"],
                         row["TRA_CDR3"])
    b = build_full_chain(TRBV, TRBJ,
                         row["TRBV"], row["TRBJ"],
                         row["TRB_CDR3"])
    alpha_seqs.append(a)
    beta_seqs.append(b)

df["TCR_alpha"] = alpha_seqs
df["TCR_beta"]  = beta_seqs

# 5) Save out
output_path = raw_dir / "IEDB_Negatives_HLA_class_I_with_TCR_seq.xlsx"
df.to_excel(output_path, index=False)
print(f"Done — wrote {output_path}")


⚠️ V allele not found: 'TRBV10-3'
⚠️ V allele not found: 'TRBV10-3'
⚠️ V allele not found: 'TRBV10-3'
⚠️ V allele not found: 'TCRAV3*01'
⚠️ J allele not found: 'TCRAJ39*01'
⚠️ V allele not found: 'TCRBV27*01'
⚠️ J allele not found: 'TCRBJ2-1*01'
⚠️ V allele not found: 'TCRAV26-2'
⚠️ J allele not found: 'TCRAJ43'
⚠️ V allele not found: 'TCRBV7-6'
⚠️ J allele not found: 'TCRBJ1-4'
⚠️ V allele not found: 'TCRAV26-2'
⚠️ J allele not found: 'TCRAJ43'
⚠️ V allele not found: 'TCRBV7-6'
⚠️ J allele not found: 'TCRBJ1-4'
⚠️ V allele not found: 'TCRAV3*01'
⚠️ J allele not found: 'TCRAJ35*01'
⚠️ V allele not found: 'TCRBV27*01'
⚠️ J allele not found: 'TCRBJ1-4*01'
⚠️ V allele not found: 'TCRAV3*01'
⚠️ J allele not found: 'TCRAJ49*01'
⚠️ V allele not found: 'TCRBV27*01'
⚠️ J allele not found: 'TCRBJ2-5*01'
⚠️ V allele not found: 'TCRAV24*01'
⚠️ J allele not found: 'TCRAJ43*01'
⚠️ V allele not found: 'TCRBV27*01'
⚠️ J allele not found: 'TCRBJ1-1*01'
⚠️ V allele not found: 'TCRAV24*01'
⚠️ J allele n